<a href="https://colab.research.google.com/github/MS-H2020/Open/blob/main/Open_Images_Object_Detection_RVC_2020_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Open Images Object Detection RVC 2020 edition](https://www.kaggle.com/competitions/open-images-object-detection-rvc-2020/code?competitionId=19205&sortBy=voteCount&excludeNonAccessedDatasources=true)

## Refference
1.   [Tensorflow Hub Faster Rcnn](https://www.kaggle.com/code/c7934597/tensorflow-hub-faster-rcnn)



## 0. Setting for Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
INPUT_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020/01_input"
ANALYSIS_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020/03_code-analysis"
MODEL_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020//04_model"
SUBMISSION_DIR:str = '/content/drive/MyDrive/open-images-object-detection-rvc-2020/05_submission'
CHECKPOINT_PATH:str = MODEL_DIR

In [3]:
import os, gc, glob, shutil, zipfile
shutil.unpack_archive(INPUT_DIR+'/test.zip', './')

## 1. Importing Modules

In [4]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from six import BytesIO
import numpy as np
import xml.etree.ElementTree as et
import ast
import tqdm
from itertools import chain
from xml.dom import minidom
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import cv2
import glob
import time

In [5]:
path1 = INPUT_DIR + '/test/'

In [6]:
df = pd.read_csv(INPUT_DIR + '/sample_submission.csv')
df.head()

,ImageId,PredictionString
0,b5d912e06f74e948,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...
1,be137cf6bb0b62d5,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...
2,8d65ca08cb5ce8e8,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...
3,4d3ad1e52ad8c065,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...
4,9b94408691c7d7bf,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...


In [7]:
df.shape

(99999, 2)

In [8]:
ids = []
for i in range(len(df)):
    ids.append(df['ImageId'][i])

In [9]:
ids[0:5]

['b5d912e06f74e948',
 'be137cf6bb0b62d5',
 '8d65ca08cb5ce8e8',
 '4d3ad1e52ad8c065',
 '9b94408691c7d7bf']

In [10]:
img_data=[]
for i in range(len(df)):
    img_data.append(glob.glob('./test/{0}.jpg'.format(ids[i])))

In [11]:
img_data[0:5]

[['./test/b5d912e06f74e948.jpg'],
 ['./test/be137cf6bb0b62d5.jpg'],
 ['./test/8d65ca08cb5ce8e8.jpg'],
 ['./test/4d3ad1e52ad8c065.jpg'],
 ['./test/9b94408691c7d7bf.jpg']]

In [12]:
img_data = list(chain.from_iterable(img_data))

In [13]:
img_data[0:5]

['./test/b5d912e06f74e948.jpg',
 './test/be137cf6bb0b62d5.jpg',
 './test/8d65ca08cb5ce8e8.jpg',
 './test/4d3ad1e52ad8c065.jpg',
 './test/9b94408691c7d7bf.jpg']

In [14]:
def get_prediction_string(result):
    with tf.device('/device:GPU:0'):
        df = pd.DataFrame(columns=['Ymin','Xmin','Ymax', 'Xmax','Score','Label','Class_label','Class_name'])
        min_score=0.01
        for i in range(result['detection_boxes'].shape[0]):
           if (result["detection_scores"][i]) >= min_score:
              df.loc[i]= tuple(result['detection_boxes'][i])+(result["detection_scores"][i],)+(result["detection_class_labels"][i],)+(result["detection_class_names"][i],)+(result["detection_class_entities"][i],)
        return df

In [15]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


### [TensorFlow, "Object Detection"](https://www.tensorflow.org/hub/tutorials/object_detection)

In [16]:
module_handle = "https://www.kaggle.com/models/google/faster-rcnn-inception-resnet-v2/TensorFlow1/faster-rcnn-openimages-v4-inception-resnet-v2/1"
with tf.device('/device:GPU:0'):
    with tf.Graph().as_default():
        detector = hub.load(module_handle).signatures['default']
        image_string_placeholder = tf.placeholder(tf.string)
        decoded_image = tf.image.decode_jpeg(image_string_placeholder)
        decoded_image_float = tf.image.convert_image_dtype(
            image=decoded_image, dtype=tf.float32)
        module_input = tf.expand_dims(decoded_image_float, axis = 0)
        result = detector(module_input)
        init_ops = [tf.global_variables_initializer(), tf.tables_initializer()]

        session = tf.Session()
        session.run(init_ops)

In [ ]:
def nms(dets, thresh):
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

### First 20 Images

In [ ]:
image_paths = img_data[0:20]

In [ ]:
images = []
for f in image_paths:
    images.append(np.asarray(Image.open(f)))

In [ ]:
!mkdir deepak

In [ ]:
image_id = df['ImageId']
def format_prediction_string(image_id, result):
    prediction_strings = []

    for i in range(len(result['Score'])):
        class_name = result['Class_label'][i].decode("utf-8")
        YMin,XMin,YMax,XMax = result['Ymin'][i],result['Xmin'][i],result['Ymax'][i],result['Xmax'][i]
        score = result['Score'][i]

        prediction_strings.append(
            f"{class_name} {score} {XMin} {YMin} {XMax} {YMax}"
        )

    prediction_string = " ".join(prediction_strings)

    return {
        "PredictionString": prediction_string
    }

In [ ]:
k =-1
predictions = []
with tf.device('/device:GPU:0'):
    for image_path in image_paths:
        k=k+1
        img_path = img_data[k]
        img = cv2.imread(img_path)
        with tf.gfile.Open(image_path, "rb") as binfile:
            image_string = binfile.read()

        inference_start_time = time.time()
        result_out, image_out = session.run(
            [result, decoded_image],
            feed_dict={image_string_placeholder: image_string})
        df1=get_prediction_string(result_out)
        z1=nms(df1.values,0.68)
        z=df1.iloc[z1]
        z=z.reset_index()
        predictions.append(format_prediction_string(image_id, z))
        data1=z
        COLORS = np.random.uniform(0, 255, size=(len(z['Class_name']), 3))
        for m in range(len(data1)):
            if data1['Score'][m] >=0.01:
                img_class=data1.iloc[m].Class_name
                img_xmax, img_ymax =images[k].shape[1],images[k].shape[0]
                bbox_x_max, bbox_x_min = data1.Xmax[m] * img_xmax, data1.Xmin[m] * img_xmax
                bbox_y_max ,bbox_y_min = data1.Ymax[m] * img_ymax, data1.Ymin[m] * img_ymax
                xmin = int(bbox_x_min)
                ymin = int(bbox_y_min)
                xmax = int(bbox_x_max)
                ymax = int(bbox_y_max)
                width = xmax - xmin
                height = ymax - ymin
                label = str(data1['Class_name'][m])
                color = COLORS[m]
                cv2.rectangle(img, (xmin, ymax), (xmax, ymin), color, 2)
                path1 = './deepak/'+str(k)+'.jpg'
                img_path = path1
                cv2.imwrite(path1, img)
                cv2.putText(img, label, (xmax,ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.9,color, 2)

In [ ]:
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:
z = load_images("./deepak")

In [ ]:
z[0]

In [ ]:
z[3]

In [ ]:
z[4]

In [ ]:
z[6]

In [ ]:
z[9]

In [ ]:
z[10]

In [ ]:
z[11]

In [ ]:
z[15]

In [ ]:
z[18]

### For Submission

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.head()

In [ ]:
df['PredictionString']= pred_df['PredictionString']

In [ ]:
df.head()